<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/Cosmos_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure-cosmos

In [ ]:
pip install datetime

In [ ]:
pip install config

In [ ]:
# CRUD operations on COSMOS 
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime
import config

# ----------------------------------------------------------------------------------------------------------
# Prerequistes -
#
# 1. An Azure Cosmos account -
#    https:#azure.microsoft.com/en-us/documentation/articles/documentdb-create-account/
#
# 2. Microsoft Azure Cosmos PyPi package -
#    https://pypi.python.org/pypi/azure-cosmos/
# ----------------------------------------------------------------------------------------------------------
# Sample - demonstrates the basic CRUD operations on a Item resource for Azure Cosmos
# ----------------------------------------------------------------------------------------------------------

HOST = 'https://xxxxxxxxxxxxx.documents.azure.com:443'
MASTER_KEY = 'xxxxxxxxx'
DATABASE_ID = 'xxxxxxxxx'
CONTAINER_ID = 'xxxxxxxxx'


def create_items(container):
    print('Creating Items')
    print('\n1.1 Create Item\n')

    # Create a SalesOrder object. This object has nested properties and various types including numbers, DateTimes and strings.
    # This can be saved as JSON as is without converting into rows/columns.
    sales_order = get_sales_order("SalesOrder1")
    container.create_item(body=sales_order)

    # As your app evolves, let's say your object has a new schema. You can insert SalesOrderV2 objects without any
    # changes to the database tier.
    sales_order2 = get_sales_order_v2("SalesOrder2")
    container.create_item(body=sales_order2)


def read_item(container, doc_id):
    print('\n1.2 Reading Item by Id\n')

    # Note that Reads require a partition key to be spcified.
    response = container.read_item(item=doc_id, partition_key=doc_id)

    print('Item read by Id {0}'.format(doc_id))
    print('Account Number: {0}'.format(response.get('account_number')))
    print('Subtotal: {0}'.format(response.get('subtotal')))


def read_items(container):
    print('\n1.3 - Reading all items in a container\n')

    # NOTE: Use MaxItemCount on Options to control how many items come back per trip to the server
    #       Important to handle throttles whenever you are doing operations such as this that might
    #       result in a 429 (throttled request)
    item_list = list(container.read_all_items(max_item_count=10))

    print('Found {0} items'.format(item_list.__len__()))

    for doc in item_list:
        print('Item Id: {0}'.format(doc.get('id')))


def query_items(container, doc_id):
    print('\n1.4 Querying for an  Item by Id\n')

    # enable_cross_partition_query should be set to True as the container is partitioned
    items = list(container.query_items(
        query="SELECT * FROM r WHERE r.id=@id",
        parameters=[
            { "name":"@id", "value": doc_id }
        ],
        enable_cross_partition_query=True
    ))

    print('Item queried by Id {0}'.format(items[0].get("id")))


def replace_item(container, doc_id):
    print('\n1.5 Replace an Item\n')

    read_item = container.read_item(item=doc_id, partition_key=doc_id)
    read_item['subtotal'] = read_item['subtotal'] + 1
    response = container.replace_item(item=read_item, body=read_item)

    print('Replaced Item\'s Id is {0}, new subtotal={1}'.format(response['id'], response['subtotal']))


def upsert_item(container, doc_id):
    print('\n1.6 Upserting an item\n')

    read_item = container.read_item(item=doc_id, partition_key=doc_id)
    read_item['subtotal'] = read_item['subtotal'] + 1
    response = container.upsert_item(body=read_item)

    print('Upserted Item\'s Id is {0}, new subtotal={1}'.format(response['id'], response['subtotal']))


def delete_item(container, doc_id):
    print('\n1.7 Deleting Item by Id\n')

    response = container.delete_item(item=doc_id, partition_key=doc_id)

    print('Deleted item\'s Id is {0}'.format(doc_id))


def get_sales_order(item_id):
    order1 = {'id' : item_id,
            'account_number' : 'Account1',
            'purchase_order_number' : 'PO18009186470',
            'order_date' : datetime.date(2005,1,10).strftime('%c'),
            'subtotal' : 419.4589,
            'tax_amount' : 12.5838,
            'freight' : 472.3108,
            'total_due' : 985.018,
            'items' : [
                {'order_qty' : 1,
                    'product_id' : 100,
                    'unit_price' : 418.4589,
                    'line_price' : 418.4589
                }
                ],
            'ttl' : 60 * 60 * 24 * 30
            }

    return order1

def get_sales_order_v2(item_id):
    # notice new fields have been added to the sales order
    order2 = {'id' : item_id,
            'account_number' : 'Account2',
            'purchase_order_number' : 'PO15428132599',
            'order_date' : datetime.date(2005,7,11).strftime('%c'),
            'due_date' : datetime.date(2005,7,21).strftime('%c'),
            'shipped_date' : datetime.date(2005,7,15).strftime('%c'),
            'subtotal' : 6107.0820,
            'tax_amount' : 586.1203,
            'freight' : 183.1626,
            'discount_amt' : 1982.872,
            'total_due' : 4893.3929,
            'items' : [
                {'order_qty' : 3,
                    'product_code' : 'A-123',      # notice how in item details we no longer reference a ProductId
                    'product_name' : 'Product 1',  # instead we have decided to denormalise our schema and include
                    'currency_symbol' : '$',       # the Product details relevant to the Order on to the Order directly
                    'currecny_code' : 'USD',       # this is a typical refactor that happens in the course of an application
                    'unit_price' : 17.1,           # that would have previously required schema changes and data migrations etc.
                    'line_price' : 5.7
                }
                ],
            'ttl' : 60 * 60 * 24 * 30
            }

    return order2

def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
    try:
        # setup database for this sample
        db = client.create_database_if_not_exists(id=DATABASE_ID)
        # setup container for this sample
        container = db.create_container_if_not_exists(id=CONTAINER_ID, partition_key=PartitionKey(path='/id', kind='Hash'))

        create_items(container)
        read_item(container, 'SalesOrder1')
        read_items(container)
        query_items(container, 'SalesOrder1')
        replace_item(container, 'SalesOrder1')
        upsert_item(container, 'SalesOrder1')
        delete_item(container, 'SalesOrder1')

        # cleanup database after sample
        try:
            client.delete_database(db)

        except exceptions.CosmosResourceNotFoundError:
            pass

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


if __name__ == '__main__':
    run_sample()

In [ ]:
pip install names

In [ ]:
pip install datetime

In [ ]:
# Generate random data and insert to Cosmos DB
from random import randrange
import random
import names
from datetime import datetime
from datetime import timedelta
import uuid
import json

HOST = 'xxxxxxxxxxxxxxxxxxx'
MASTER_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
DATABASE_ID = 'xxxxxxxx'
CONTAINER_ID = 'xxxxxxxxxx'
Number_of_docs = 50
regn_range = 50000
d1 = datetime.strptime('1/1/2008 1:30 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2009 4:50 AM', '%m/%d/%Y %I:%M %p')

starttime = datetime.utcnow()
print("Starting ingestion: ", starttime.strftime("%Y-%m-%d %H:%M:%S.%f"))
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

def Create_Record(item_id):
    doc_body = {'id' : str(uuid.uuid4()),
            'employee_id' : id,
            'first_name' : names.get_first_name(),
            'last_name' : names.get_last_name(),
            'regn_no' : (randrange(regn_range)),
            'start_date' : str(random_date(d1,d2)),
            'salary' : random.randint(12000, 2000000),
            'inserted_at' : str(datetime.now())
            }

    return doc_body

def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
    try:
        # setup database for this sample
        db = client.create_database_if_not_exists(id=DATABASE_ID)
        # setup container for this sample
        container = db.create_container_if_not_exists(id=CONTAINER_ID, partition_key=PartitionKey(path='/id', kind='Hash'))

        for i in range(Number_of_docs):
          document_body = Create_Record(i)
          container.create_item(body=document_body)
          if (i+1)%100 == 0 or (i == Number_of_docs-1):
            print(str(i+1) + " documents completed")

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
        endtime = datetime.utcnow()
        print("\nrun_sample done :" + endtime.strftime("%Y-%m-%d %H:%M:%S.%f"))
        print("Time taken :" + str(endtime-starttime))

if __name__ == '__main__':
    run_sample()

Starting ingestion:  2021-11-17 05:22:11.511193
50 documents completed

run_sample done :2021-11-17 05:22:25.200998
Time taken :0:00:13.689805


COSMOS PAGINATION

In [ ]:
import logging
from azure.cosmos import exceptions, CosmosClient, PartitionKey, documents, cosmos_client
#import azure.cosmos.cosmos_client as cosmos_client
import datetime
# ----------------------------------------------------------------------------------------------------------
# Sample - demonstrates the basic pagination on COSMOS
#
# ----------------------------------------------------------------------------------------------------------

ENDPOINT = "https://anucosmosx1.documents.azure.com:443/"
MASTER_KEY = "yxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
DATABASE_NAME = 'testdb'
CONTAINER_ID = 'testcoll'

#client = CosmosClient(ENDPOINT, {'masterKey': MASTER_KEY}, preferred_locations = ['Central US'])

client = CosmosClient(ENDPOINT, MASTER_KEY)
database = client.get_database_client(DATABASE_NAME)
container = database.get_container_client(container=CONTAINER_ID)

def query_container_1page():
#   query = 'SELECT *  FROM c WHERE c.Legal_Entity_Customer_Number = "CA00536503" and c.transaction_date > "2019-11-01" and c.transaction_date < "2021-01-30T00:00:00"'
    query = 'SELECT *  FROM c WHERE c.name = "Anupam"'
    t = datetime.datetime.now()
    items = list(container.query_items(query=query, enable_cross_partition_query=False))
    t1 = (datetime.datetime.now() - t)
    print("Time for query :", datetime.datetime.now() - t)
    print(items)
    request_charge = container.client_connection.last_response_headers['x-ms-request-charge']
    print('Query returned {0} items. Operation consumed {1} request units'.format(len(items), request_charge))
#    print(container.client_connection.last_response_headers)

def query_container():
#    query = 'SELECT c.customer_duty_total_exempted,c.customer_effective_discount_total_net,c.customer_effective_discount_unit_net,c.customer_invoice_price_unit_net, c.customer_invoice_total_gross,c.customer_invoice_total_net,c.customer_invoice_total_vat,c.customer_retail_value_total_gross,c.customer_retail_value_total_net,c.customer_retail_value_total_vat,c.customer_vat_total_exempted,c.delco_retail_price_unit_net,c.delco_retail_total_gross,c.delco_retail_total_net,c.delco_retail_total_vat,c.Legal_Entity_Customer_Number,c.quantity,c.vat_percentage   FROM c WHERE c.Legal_Entity_Customer_Number = "CA00536503" and c.transaction_date > "2020-11-01T00:00:00" and c.transaction_date < "2020-11-30T00:00:00"'
#   query = 'SELECT *  FROM c WHERE c.Legal_Entity_Customer_Number = "CA00536503" and c.transaction_date > "2019-11-01" and c.transaction_date < "2021-01-30T00:00:00"'
    query = 'SELECT *  FROM c WHERE c.name="ANUPAM"'
    t = datetime.datetime.now()
    items = list(container.query_items(query=query, enable_cross_partition_query=False))
#    items = list(container.query_items(query=query))
    t1 = (datetime.datetime.now() - t)
#    t = datetime.datetime.now()
#    t2 = (datetime.datetime.now() - t)
    print("Time for query :", t1)
#    print("Time for print :", t2)
#    print(items)
    request_charge = container.client_connection.last_response_headers['x-ms-request-charge']
    print('Query returned {0} items. Operation consumed {1} request units'.format(len(items), request_charge))
    print(container.client_connection.last_response_headers)

def manage_provisioned_throughput(ws_throughput):
    print("\nGet Container provisioned throughput (RU/s)")
    try:
        # read the container, so we can get its _self
#        container = db.get_container_client(container=id)

        # now use its _self to query for Offers
        offer = container.read_offer()
        print('Found Offer \'{0}\' for Container \'{1}\' and its throughput is \'{2}\''.format(offer.properties['id'],container.id,offer.properties['content']['offerThroughput']))
    except exceptions.CosmosResourceExistsError:
        print('A container with id \'{0}\' does not exist'.format(id))
    # The following code shows how you can change Container's throughput if it is in manual mode
    offer = container.replace_throughput(offer.offer_throughput + ws_throughput)
    print('Replaced Offer. Provisioned Throughput is now \'{0}\''.format(offer.properties['content']['offerThroughput']))

def query_with_page():

    query = 'SELECT *  FROM c WHERE c.Legal_Entity_Customer_Number = "CA00536503"'
    t = datetime.datetime.now()
    query_iterable = container.query_items(query=query, max_item_count=10, partition_key='CA00536503')

    pager = query_iterable.by_page()
    newpage = list(pager.next())
#   print(newpage)  # print first page
    print(len(newpage))
    print(container.client_connection.last_response_headers)
    print("Request charge 1st page" + container.client_connection.last_response_headers['x-ms-request-charge'])

    token = pager.continuation_token
    print(token)     # print the token after retrieval of 1st page
#    pager = query_iterable.by_page(token)
#    second_page_fetched_with_continuation_token = list(pager.next())
#    print(second_page_fetched_with_continuation_token)  # print second page using token
#    print("Request charge 2nd page " + container.client_connection.last_response_headers['x-ms-request-charge'])
#    token = pager.continuation_token
#    print(token)    # print the token after retrival of 2nd page
    return token

def query_2nd_page(token1):

    query = 'SELECT *  FROM c WHERE c.Legal_Entity_Customer_Number = "CA00536503"'
    t = datetime.datetime.now()
    query_iterable = container.query_items(query=query, max_item_count=2, partition_key='CA00536503')

    pager = query_iterable.by_page(token1)
    list1 = list(pager.next())
    third_page_fetched_with_continuation_token = list1
    print(len(list1))
    print(third_page_fetched_with_continuation_token)  # print third page using token
    print("Request charge " + container.client_connection.last_response_headers['x-ms-request-charge'])
    token = pager.continuation_token
    print(token)

#   MAIN PROGRAM
#query_container()
#query_container_1page()
token1 = query_with_page()
#print("Calling next page")
#token1 = "+RID:~esxdAP7UUvi6hQEAAAAACw==#RT:2#TRC:4#RTD:0lMqZknzxUA8EnGqBD+cBTMxMzEuMjEuNDFVMTE7MTE7MTEA#ISV:2#IEO:65567#QCF:3#FPC:AggAAAAAAAgAAAYAAAAALAAAAAAAAAAIAAB8AJKA2IDRAEAIf4CJgkuBWoAkgQeBF4Nng3CAcQFACD2BZYJhgUWFo4AogV6AFIF8gfaAfoBZgLSAM4C7gKWAAIAEgGKAVIB5gVCBgIAjgJ6ACYBsgKKAg4JCgkaAaYAOgN2AtoATgKmBTID4gIyAhoDegHCAaYCFgeuCwoEBAAAAAAgAAHAAU4H2gRyANITRACABMQDAAI+A7IGMgNuB9oAsgE6AAYCDgQeAxIAmgDaB94JcgGeCHoOigK2BCoLagG+BC4Z1gBiB+IBIgMmADoBvgK2AZIF9gKKBJoAWgbuBV4CqgVmA4YChAAGAPoAYgL+DUYKYgAIAAAAACAAAfADFgC6CA4LtgbGA1IAUgaWCT4AnhJmAX4LJgFqAnoHbgHyBU4AqgLiA0YEdgUOAHICjgNaB0YKTgAeAmIBggDyA74CVgE6AR4GDgVOAQoDOgMqBHIBEgHCA04GSgkiAJYIOgTaAHoCWgJqABIB1gOOA1IGugC+ADYAfgP+BAwAAAAAIAAB0AO2ADIAzgiOCNYASgByAFIH2gfeAl4DNgR2AA4DCgDaAwYCmgguCGIF8gr+AgoBigNSBZYBxgQKAHIB9gYqAfYJUgdSDUIA5gBCARYAJhHiAF4BnguyA/4BtgQ+DD4A1gp+A3IClgCaDJ4A/gQOAqYAUgICABAAAAAAIAABcAC2AfYGUgh6A/oSIg9EAEAgpgomE/ICBgICAJIARgP6AboFwgAuAC4J8gHKChYAWgdyA24DpgFSBE4CpguyBWICmgWmAmoAIgtOE/IBpghOACIC4gauAr4IkgAaBBQAAAAAIAABeAG6AG4BugV6AG4CMgK6ABYD3gBuA1IDZgmCAsYFKgV6B2IFMgwSAm4ADgGGAZoD4gbSAVYOphCGABIiQgG+ApYBqgMmBqoJPgFOBzoBfgD2A9oDpgM2AAYS6gBeCI4MGAAAAAAgAAHIAXoGNgDyBH4DTgcOB0oEUgbSAJ4ABAQoA34EfgTuBA4DNgRGEZIBmgECAcYFFgA6BOYBtgVmAjoBSgOyBUIEPgK2AwIEMgDyB/4BJgxOCo4CCgXeA5YBhgISA34NJgOuA4QAhAA+C7oAZgMiAEYJLgKKCBwAAAAAIAABaAAKFBoB3gMaAjoIkgVOAJoEBwAAMMoC5gBeBrIE7gECApYCZgPqAmYFygaCAzYEMhPuCj4HUgsmAeIAvgMCDRoAHgAmA2IDRhQOBdIMcgKuAAIATgKKAdYPVgAgAAAAACAAAcACOgBSCcoAdgR2Aw4LXgaCAFoHygcmAGYD4gY2AYYJagKWAHYB/gPGAfYHEgTmAqYArgHiApYGlgleA2oFfgTmAHICAgLeB2IC9gHGBsoB8gDWAvYDKgDaB0YHfgnKBJIKhgD+AM4GQgJuA/IAIgx+BCQAAAAAIAABkAByBw4JFgyiA84CXgMOAVoE/gReBQoBJgWCBnoDLgPKAPIBwgMCAMIDqgJCB8oXqgCqA4oASgHGAK4C7gOKA2oBYgLaBz4BtgLKA7IBlgTuANoKsgoCCkoGxgPOBpoGLg5yCnYAKAAAAAAgAAHoAXoIZgAKBH4BggT2BKYAegWOAcoDTgVCANoGigIuA0YHVgQOALoDogHWEP4GugKmBp4GGgI2BToB6gU2AhYBUgrWAYYV1gSiABoBKgMSACYKqgXyAyoBjgJqAEYAJgICAjoG4gJuAO4AagB6Bq4ITgNeBCoCdgc+AZ4ALAAAAAAgAAIYAV4CRACQAt4Aig1iAKYA4gGOAR4APgDaAAIBFgOqB74KLgU2BZIEjgCaAQ4JggQ6AA4AcgL6BN4MlgaeABYCagBGAG4DSgt2A0oF/gBOAboC1gKCAVYAdgKSABYAGgRiCWYNGgWiAlYA4gJ+B24PLgC2AxYCGgBKANIAWgNaCUYHMgDmBEoAMAAAAAAgAACgAeoBsgbmAAoCUgGCBmYEigTmEnYADhDyAgYF+gCGEA4AxgQ6AsQAQEAAAAAAAEAAAMABZgEmDgIpEg+eAJIYzgr+CSYAygDeA7IBXga6AmoRcgN6AdYNOgGKKd4LcgayA3oMBAAAAABAAADwAYoA6gTqAqoBRgwOBK4IJhbaAUYIWgKWAeYQJg3CCA4RigcqB14GEgRWACYBaibWB9ILFgZ6CIYDcgCKAAgAAAAAQAAA0AKeCa4DBgU2Bb4Btgw2CjoEbgbWDjYNQge2DKIGkgMWCnoQngCaGsoJRCkAEIYMIguCAR4EDAAAAABAAADQAY4Cbgf+FI4JxgD+Gc4H0gA2BFYPJgx2BM4BSgRGAW4nrggaAroM1ggGAKIGPg1KC8IMigAQAAAAAEAAANACjgJqDtYFFga2EOYLKgKeAx4WeghqAdYA/gDqCeYF8hMKB8YfThK2CpYAfgDKDwYKUgLuABQAAAAAQAAA0AJ+D3oLHhDmDDIJvgTqAIIE2gDiFYIBdgIKAToLdgAmApYLsgdyAzIPsgoGGY4WfgmeDjoAGAAAAABAAAEAAMoD6gIuAsYBhiEGAZ4L1gGeBF4E6gCKBkoH3gYOB24A4gHuAl4RChBqFO4CkgSqBAQEMAN6Aw4A5gcCEpICDhgcAAAAAEAAAMAAkiIOCpYB3gImLn4J2gMSAnYBGgaWBaoEKgKaCeYTNhfOBLoGEgYCAN4C7hC6DUoAIAAAAABAAACwAD4CahWmIyIEUg2CCaoAHgzyBEoK7g+mAn4c7hFeAtYCOgLyG3YArgW+ArIEJAAAAABAAADgAcIMNhEOBa4XHgCmBAYF4gyWFTYEJgFiDZ4MygIeDfYAQgLeBRYBLgruAm4b5gL6DIoE1g5aDoYAKAAAAABAAAD4AJYI+giCBpYCpgHOBCoC5gCuAYYJ2glmAEIAlgbeAGYL5g1CCwYYHgo+AzYH8hA6FWoKJgAWCboApgDeEg4ILAAAAABAAADIAw4B4gLyBYIDfgRaDmYOoh8qAAYRXgLKDcoGPgoEACEATgBCDUogNgGqCh4PRgCGBm4oMAAAAABAAACgA94GxgdGCnYNagF6CpoV+hgaBEoOVgLaDY4PBhIGL34DOgF6CrIEWgQ0AAAAAEAAAKABqga2ACYH0ifKB6oHggsCHM4D2gpGAZYOigB2EI4LLgCCF2YidgbqBDgAAAAAQAAAoAJqB+Yi7gROBpo9Wg0yCOoHLilKCY4AqgJaCjoDEgFqDHIBOgD+CuoAPAAAAABAAADwAgIHBhLuBWYAggdaEa4CNgduBVYIIgbmBkIDggiKBQoBvgeiDOoFLjFeANIBDgouBK4ElgYWAEIAig/yDEAAAAAAQAAAmAB+DBIRcgA2KyIC/guyB0oJPheWEwoKfgR+C4YPbhnmAw4EogMKBEQAAAAAQAAAyAMqAzoDuhEyDSIPKgBSD5INSgGqKB4BYgJ6CjoGtgDGAKYGchwGB6oH9gnqBGIDjgu2BEgAAAAAQAABAAGiE8YBlglSBCIGIgB+AA4e2gC2CU4FDgZmIl4IlgH2AZoOKg8qDzIBOguGBBYP2gJqAQIBCgISBd4F8gQKBHYETAAAAABAAADAAu4HIgEqFooGEgieBf4QAiv2BnoTUhZKBLYEHhIOClYFPgCaAYIaZgY2AN4BegAqCFAAAAAAQAAAuACyCXIEigrKCMoBcgdODl4NvgviGgYVkhJaBZoHdgVyEL4JMgvCDloJGhT6AtoAVAAAAABAAAEQAPoIMgdaAP4AIgTaAWICthiiDR4CWgB+CroE6giODiIKtgD6CCIFtgtCAOICCgJiBWoC2gTWCMIShgVmBq4BRgEiA54cWAAAAABAAADQA8YAsgISBeICLgHCBBIDMgT+C6IgmgrmAM4CWgp2JUQIgwD6LlIGGgaCBmoUWgDaBeIB3gxcAAAAAEAAAPgB5gU+FdYBMglWCC4Q4g2qDq4ECA0ACQABKgXWBK4Hrhs2AuoCKgF2A84JkgCCB+4IviEiBCoApgJKABYLCgBgAAAAAEAAAKACLhIOCZIQggRiAH4eZhGWBX4F4heyFDoDPg9uGX4G+gbGCKoChgEOBGQAAAAAQAAAwADiCaIawgBmAEoAiguCDz4ENg5SKZ4EMhb6AxYD6gG2DXYNTgnaAGoD2gdiDkIEGhRoAAAAAEAAANgC2gziDwIBSgUWAVoBrg4OB+IFWhFaCr4XzgmqAqICOgCWAkYE4gHCDaIOmiS6CzYIRhO2AOIAbAAAAABAAACAAEoSMgMKA9oOIgJqCwYPkge2H5oIbhM+CYYF5gsqDyIUAAAAAACwAAKwAjYAegJaBm4HJgBeBJ4AHgDiAKICFgHOAW4DNgAuAa4FqgNEAAiAWgUmAO4D1gbaAD4AngUCBNoCDgEuAZYCggMOAVoCngFeAloBPgEeAiYBPg2+Aq4F0gPuBhIBXgjmAVoBOgPCBsYAEgLWBTIDHgNaByILWgBKAdoDlgCOAQoB6gKyAe4IxABAgjIAtgIyBN4BIgQSAUIBLgJuB5oBRgA6AvYBTgACAAoBGgAEAAAAALAAA1gAHgHuAV4D/gJuAaIC7gEyAQYB2gKSANoAvgEKACIB9gGeBGoBVgEaADYCSgQSA+4GBgBKAcYAQgJGAKoDLgJiAB4AugDqA0QAAChmA9IAtgHGAZoB5gAKBE4DYgISAaoBigLiAc4CQgLiAFoDOgHaA0QEAoFCA5oAygGeADoAagUmAhIEbgBaAV4AegZSAhIAagFOAx4BbgLyBroA0gAWAToA9gDqBbYDvgOSBnYAHgSWAMYP8gDeA8ICagByAK4ISgFSAIoE1gDaALYBBgSiAf4CJgIaAAgAAAAAsAADOAHKAIoBDgRGC7IAogL2ABIBLgNSBpIAigD6AJYEigAGB9oACwIEAgAAygDyAIIC9gA6Ar4CDgFqAfYAAgOWASIBbgO6AfIGNgFKBU4A1gHmA4QCBAKuANYAVgEiAEYEogFuBJ4CSgDEABAE2gLmB1oGQgJOAvYAvgb2AUoDbgCeBc4CegFuBioBwgNOBMoAdgOKBGYD7gH+A8oBCgICBXYBqgGqAU4A6gGyADoBZgNiABYCrgLGAVoA5gCWA94APgC2ADYDHgDmBzYApgEGBAwAAAAAsAADAAL+AAoAKghSA7IDAgPSA/4ADg0eAnIEMgB2AQIARAwkAtoD+gE6AJYFJgN+ASYD/gJuAAoBKgAeABoAqgCGAXYDvgBWAXIAAgRSBHoAXgAGAIoAugBEAIIAygKEAAKAogLKAA4AIgGyCMQAgIMEBCgBigGaBMYA2gBmAAIDkgUKBqoGagIKAcYJwgGaBFYACgMmAXoBPgICASoH2gFuAyYB0gH2AB4AGgNWAUIEFgIKAToCigA2AOYCZgD+CV4HngAQAAAAALAAA0gAGgB2BE4B9gJ6ABIAhgZ+AIYFVgCEAAQSYgRqALIDvgK6A34NUgByAPYAegDaAH4AWgAuAZ4BogD+AA4B/gMOABIBxAYBAEoAfgFGAr4C1gtiAG4BsgJSAdoE0gBOAf4FHgAKAHIDFgKmBlYCvgEaBYIALgBWAI4EWgBqAHIH5gGqASICBATAAfYDHgqeChICOgL2AQoAUgGqAUYBKgKCAWYF8gUOAJoCqgOSAxoCBAEAISYB+gHGABYDNgfMAIAQABAACZYArgH2AS4AOgH+AA4AFAAAAACwAAAwGG4CIgL6AHIAYgASAAYAXgAaAVYBVgLCAcQAgAlCALoATgLqAOYDUgBaAb4DQgCOAGoAAgI6AEoBtgIOACIADgTEAIAglgQaARYBngFOBH4A2gEmAIYCFgCWAOIApgEeAN4BbgAuAG4A+gBSAG4ACwABBgAAWgAHABAgqgCmAHYBDgAiAG4AJgAyAL8CqA4wZGAHghuLTq4Tm3iOoARYwFUWAIAIigVBSxJlDhgGgVEgQNFJUCgMQAKAlBkRQOoQ2wBBIpkDSbQpRFeAgogoOAqgCuFWBDcwFrEclmlQQBMUEQoUZiqQHAkAEE4APgBEAAkAVgCXAUPAiAZBIlRISB5CkKZZEIUkICkjiQZAQRMEAAgAChADaGRACAQAoEYAJAECAIRIAAARAAIQABAFKAkgAEGwAJAICIAAGALBJAggfgCTASLSKoQCigCCBAwggQgAAQKAABEiARAQCiYEEIG4otEyIAB4riAuGABosgEADYAAIpKEQANgAAGBkI4AkoJxWODJSyAAAAAKAHYA+wgHl+A1DAL1KjvwHrBGCXwOwgwHlqA2gAVBAUUuh8APBIEhHO4JuxABAASQBED0g7GYwqwTAYENRcxIQIYlCwiEB2BA0RgDLBdkoAW0CFFCQIAREQGqgfIjNFEQYfDFNkYuhEvjgoAgJbUAi0wDABKYrIgEHKEAIRdhGYBHsIGICyqIQmyNLlQMJjxBQU4iIGCIeBBCIEYRFgLOYYGCvJDBX0JDtSPi4NAJ36QQCoNGEGIGwNUDRAqYYQHJJ2iVUEEiA4VIgYRNQEQqJAWQLkIQSEJSKmjEMgkuQBElBtYFWzxgBDNB5pjAwVCuwRAFEXGUAE4iEHA0CkIlq4SYjIoUBlIGNIIJMQoeDVHlSjoY6y1KkQhEFhiM2mRAZRgYAFmAmAcmaJTEisTIFbiEFylNgAXQ46qAsqTSiHiALwYKCwAGULR7bg4W0BhKwDAW0bYoBxAuTACERBAAewFwWAPiRlJIiLY4GIC70AWDGSbwYmAyghBnyg4IcgACwqq2DJMcCIAAmAQCkFtJogBIIIXMkEGPgCBbVAEBAwEBQIvAACgKUDijXyEEgYdCaEZAKEjH0Y0BDjAQEcPAQgwyCEmwSGjSkG2yC6AHGABEOJYAxIcYUsgAkAEippgErlYHZiAgUNxo4Bk2NCmKRAKSYiRmUykJiB6ZK+SJQAsEmJDEDYHQgwNRSAAMAmSOArIABARBzgRMVyZBKxBjAIw8SgKDDKBqSgeI08AgUjJiQAgCyIsTgSsg9ANnghAUKlrwFAwjcyIFAEAn5CKIgLgghrGBhQsQogJUxQoJQXHiFtMhGa5JMImJZkwAAwJaYElDhGuB/ymxU/hyPpBZ2Fkk6w97w1izCgghAYQ+RCCFMCGAG3LGQBBAjhCcUFE6IWUPeHS2+Lqy87jDCOtA3OpKACHneFixEReLHZg0XyPBJ6aFEmOMuz+l0hlcKoyKtNTEQoqBUhkuNrBZocJQIWDpCBw3EBsDB4KiRqQa+BlskUaKwiiDUKURIcBkCZADGII4EAsHqhBMITgaqAwhGJUZxVSUKEkiU+ODA5uiruAhZE9oGCLp4F0kbgwjhlogpCDwAA7NABRJA2DFkLKKlKlAiRkIFGgxojKdBAHM6G12BsA3SSGAYcxjoUhcw1aKRAFgTbGEhfoAgILEYjgSDJhLY4SEAgLbtASSwEAYY0aRCza0gY/wA6JYhGESalGKkDQcTIbgEF0AoIalzCoQqE0ALYCpMsgS4CBIBAbRQgzQbhSA4E4CAiiHN7a7SxGUAE4SNCgFZCLmwQhBZ/tgYAyQEN6kOZL1RsN6SNEEQLsKXECIPAsqGgh24iuBIMQyhDwHinCSAQlw6xwBQFLANTISikgCKjB0QnCF0mQBzDkBSYaDay3ISRgKTFEtAl0CotAOFPEPCq8oO4TGMRFBbTCBNNsAj8EBxAKFmKapUlIil4HYRKUgGlhrsWiAFI0DhBSjhcU+TpEaZZJkEUjgGEUlSIuSCEFIBhA5DoOACoBKyqBWvQUeMfBls4JS4SqEABgAAAAAsAACMBj/DOmQEsGOxqkQkA4JFhjUFCjUpEGQMMsSUfkE7YQCBGgEJLPE6IyExhgnSmABIKbDWlj1YhKEYjCUtIB8FYmPEVAkiChIYIAjegK3PIak8QKCTEFQi4bHFgAIKiTZgppSXQIQFjGEjXIEKAYDTA5oYQlXI1Ep2SOXBCqYR8MtUAnKXYYBSDUxAIUqe3uFOYLsELAQiAAZRFIZCAUTBxiYmpaxuUiKlkBITRAhFLnDGE7gGpaRIrIe0BWhUFBADEq2EUi5oQMTpOCDVwDzkZxQhVphiELkUVQiLOJEKAcgxoARkgAXFBMCjhKhgIAFozIHYWCAFyCPejjgAC6gJQAKlBKoRAxkFPIyJEhy/EZS7FylnLJcGMawVKoIRASZNXDqgwrRIBbwh0AixSRbCAhZCDbwwAQRiNGmgTowjYAAQGJEAcEAEFQxIIA2AoBQnRJ3C+gkkOEClmUBVAAUAQVPwrSFoUGhha4p0FJQCkacuQBJBqwGjPA4oiZkCByDQHSAzpBSXImyUFUPAXAVJCKFcHDSOBQMoYmG9xkwUdER2GqnYIMLkRpTSh1yBIMM4MNwhBAJAAgEBCgEQB0VAKwEVIUWl6SRMZmwQsYBUI1M0iTKCCS+oMBlWCGGxgKLrQgUgHbA+QjTdYxWNWAhRI0KGyZ5K58vLkgqaOsEsLAegFEkkAIbRRCfogPALxeiLqMEEGh8HABAmrBEoistIoTlgUFDQAQwMghBgVkA2hygoD+4SY0DCopAjiohRVItQqBe8cokCmAIBaNFYEXQ5uIxwy0NAg5QEEpoAvG0RAAEaJEziobiAXzGR6lgFEspkEmA6BJAhmZBmoUoCEJ/BUhAbbqDQAMAxURZFgAFELJhBMADKAIKYOK0HJAEgr4UBzoobbhYAQOLSAgQglGkwJFC4tiPqAxSXMKHRAAUvovdsGy0YcKBS1BgAw8AfgCQFjDoQRe75A42KgS8epwFmrQhGLBPowquNWojCXiBC4VYCUhArQEARmQngMXAKTpYHgQBQChO0wJgHEACSxZA64YDGgSEiRCADCK29VkAwpS2GKeMAeIOER2D4rE4BJgCgAZ3DBG4vECHCiEaqCRwGjwMPANggAiAEBAy1RigIGmAr7gpgAKYYFspBwAEMgk1YPkucFJIkBjIEMJpRQPjiCEEDQcWYxYaWFCYVTEgEESJRRM0pbiLIYUAGEBQGQ4BgRsBODRSANA8MeMgZOgYRIBC+WANBREKEIUjt6bnAGSGgjFJCAC5MKqmyVWgkkwACnQIBKMAt0DhhBBNYQvFANAgTFi8GmEBYWHyMOmhYoIloY1QiQLcoAASOxSO0QyXMGsiAviIEKxIIIg2QYYC4hEIAhwlkM3D5UC6pYFMAIBamkAkVVBgQWBA4AAEESCZmAODD8ZZVhi9OwQBvcgEoJCVQDAgAubKCBwBMAUEQiAkCApaJMkRFwZhYBEPAkZp4RGZAGhwkJMJ8QIFOoQUAjKsBoVCJBEBIF42ABvWRADRRasNEDAtASnBp0Dgktt2OESQ0ngAN0jrFqYQEEhOSfkFqrYE5jpDJOQAYQQp2QIQhshCZMMHMXxJIag/ZME4AA6BCRoKABPtLOZPCOGQPjCTIEOCYEAjUEGDTwnwADjZEBVqCBhpBdXB5GoR7vgRNUuFNxIQSQpAq1PQYVAEIrYmKOz0UmLtiKkRKCmQYABAC0ghAhNdSVLkerrXC0qXazTUDiAlKbOMILAsYhgU4pSC1zymRAAQkgBSbEDCKilQUBYmEPDTGqUQEIIRishXGEyZMEkazkiAEKdZKJxU6mR4RQYgpJiiNmhXLgB1QdQhcGiqASA8wDWoQTJQhIkPSQwQADRgBGAhiZSzIIk0gYrhtlmxiAJQKQAISDApGXGIUEhsHUINrgRUBEhA1EKAaIIIgsoJeAeEZgMBFOPYCDrjvaegkBMMXJIINkAEAQEA9L2A0CAterkCDYGqiyICgEDMgRPHWgkylSAMAAGCFScAMoEzJgKDIWvLC0CBO5YcAAwfKYKEkCalCQgdIBYNJ8JpDIzAE+jAX0FI0hKlxCCHYocENiASTeiFyHJ4iXYIDKAyhuBIAA4QIyJohCsYIUKaGaMPOkSBIMwAsSZqAMUAIiqDQiPR7HaOVIRcQKABDAZAoTMv0gGoDUIUjglZAWQySOHAIkIGxRJhNZK60CDEWRqSBIAjBkThUADLCRQIAK4AygDEAMABfgAaA"
query_2nd_page(token1)
#manage_provisioned_throughput(100)